In [112]:
import pandas as pd
import numpy as np
import os
import sys
import shutil

# --- Configuration ---
input_base_file = "sequence_dataset_0.csv"
output_file = "sequence_dataset.csv"
duplications = 100    # Number of times each row should be duplicated (Original + 5 noisy variants)

# AGGRESSIVE AUGMENTATION PARAMETERS
JITTER_STD = 0.05   # Increased standard deviation for random noise (was 0.01)
MAGNITUDE_RANGE = (0.9, 1.1) # Factor to randomly scale sensor readings (simulates force variation)
# --- End Configuration ---

# Set random seed for reproducibility in augmentation
np.random.seed(42)

# --- Renaming the input file to the desired output name ---
if os.path.exists(input_base_file):
    try:
        # Using shutil.copy ensures the original source file remains untouched
        shutil.copy(input_base_file, output_file)
        print(f"File successfully copied from {input_base_file} to {output_file}.")
    except Exception as e:
        print(f"Error copying file: {e}")
        sys.exit(1)
else:
    print(f"Error: Input file '{input_base_file}' not found. Please ensure it is in the same directory.")
    sys.exit(1)


# Define columns that MUST be treated as strings/objects (no augmentation here)
label_cols_to_force_string = [
    'digit1', 'digit2', 'digit3', 'digit4', 'fullSequence'
]

# Create a dictionary to force string types on load
dtype_spec = {col: str for col in label_cols_to_force_string}

# Load the (now copied) dataset
try:
    df = pd.read_csv(output_file, dtype=dtype_spec)
except Exception as e:
    print(f"Error loading CSV file: {e}")
    sys.exit(1)

print("Original dataset shape:", df.shape)

# Base sensor columns for one digit (12 columns)
base_sensor_cols = [
    'accX', 'accY', 'accZ',
    'gyroX', 'gyroY', 'gyroZ',
    'rotX', 'rotY', 'rotZ',
    'magX', 'magY', 'magZ'
]

# Generate the full list of sensor columns for all 4 digits
sensor_cols_4_digits = []
for i in range(1, 5): # Digits 1 through 4
    for col in base_sensor_cols:
        sensor_cols_4_digits.append(f"{col[:3]}{i}{col[3:]}")

augmented_rows = []

# --- Aggressive Augmentation Logic ---
for _, row in df.iterrows():
    # The original row is the first instance (0 noise)
    augmented_rows.append(row.copy())
    
    # Create the specified number of noise-injected duplicates
    for _ in range(duplications):
        new_row = row.copy()
        
        # Apply aggressive noise and scaling to sensor features
        for col in sensor_cols_4_digits:
            # Check if the column exists and is not NaN
            if col in new_row and not pd.isna(new_row[col]):
                # 1. Feature Jitter (Random noise with higher std)
                jitter = np.random.normal(0, JITTER_STD)
                
                # 2. Magnitude Scaling (Randomly scale the base value)
                magnitude_factor = np.random.uniform(MAGNITUDE_RANGE[0], MAGNITUDE_RANGE[1])
                
                # Apply both transformations
                original_value = float(new_row[col]) # Cast to float for calculation
                new_row[col] = (original_value * magnitude_factor) + jitter
                
        augmented_rows.append(new_row)

# Create final augmented DataFrame
df_augmented = pd.DataFrame(augmented_rows)
print(f"Augmented dataset shape: {df_augmented.shape} (Original rows * {duplications + 1} total versions)")

# Overwrite the output CSV file with the augmented data
df_augmented.to_csv(output_file, index=False)
print(f"Dataset overwritten with augmented data: {output_file}")

File successfully copied from sequence_dataset_0.csv to sequence_dataset.csv.
Original dataset shape: (50, 53)
Augmented dataset shape: (5050, 53) (Original rows * 101 total versions)
Dataset overwritten with augmented data: sequence_dataset.csv


# 1-digit dataset

In [113]:
# Import libraries
import pandas as pd

# Path to your sequence dataset CSV
input_file = "sequence_dataset.csv"  # Update path if needed
output_file = "predict_digit_dataset.csv"

# Read the sequence dataset
df = pd.read_csv(input_file, dtype=dtype_spec)

# List to store flattened rows
rows = []

# Each sequence can have up to 4 digits
for idx, row in df.iterrows():
    for i in range(1, 5):
        digit_col = f'digit{i}'
        if pd.notna(row[digit_col]):
            # Extract sensor readings for this digit
            accX = row.get(f'acc{i}X', None)
            accY = row.get(f'acc{i}Y', None)
            accZ = row.get(f'acc{i}Z', None)
            gyroX = row.get(f'gyro{i}X', None)
            gyroY = row.get(f'gyro{i}Y', None)
            gyroZ = row.get(f'gyro{i}Z', None)
            rotX = row.get(f'rot{i}X', None)
            rotY = row.get(f'rot{i}Y', None)
            rotZ = row.get(f'rot{i}Z', None)
            magX = row.get(f'mag{i}X', None)
            magY = row.get(f'mag{i}Y', None)
            magZ = row.get(f'mag{i}Z', None)

            # Append a row for this digit
            rows.append({
                'digit': row[digit_col],
                'accX': accX, 'accY': accY, 'accZ': accZ,
                'gyroX': gyroX, 'gyroY': gyroY, 'gyroZ': gyroZ,
                'rotX': rotX, 'rotY': rotY, 'rotZ': rotZ,
                'magX': magX, 'magY': magY, 'magZ': magZ
            })

# Create new dataframe
digit_df = pd.DataFrame(rows)

# Save to CSV
digit_df.to_csv(output_file, index=False)

print(f"Digit dataset saved to {output_file}")
digit_df.head()


Digit dataset saved to predict_digit_dataset.csv


,digit,accX,accY,accZ,gyroX,gyroY,gyroZ,rotX,rotY,rotZ,magX,magY,magZ
0,7,0.903889,5.289334,8.043473,0.382707,0.074067,0.237626,0.170200,-0.233344,-0.729515,9.675000,34.200000,32.925003
1,0,0.987039,6.090930,7.283752,0.329867,-0.017868,-0.021380,0.187426,-0.257631,-0.722812,11.643750,29.643751,25.181252
2,1,0.797408,6.619744,7.086943,-0.013286,0.308029,-0.062461,0.206066,-0.260063,-0.721460,15.862500,20.662500,11.606251
3,7,0.970664,5.386789,7.320857,0.382707,0.074067,0.237626,0.170958,-0.132010,-0.759705,9.035858,32.061615,32.488959
4,0,0.850162,5.787082,7.104787,0.329867,-0.017868,-0.021380,0.249451,-0.269653,-0.765578,10.882029,30.705991,24.865260


# Predict 2nd Digit Dataset

In [114]:
import pandas as pd

input_file = "sequence_dataset.csv"
output_file_2nd = "predict_2nd_digit.csv"

df = pd.read_csv(input_file, dtype=dtype_spec)
rows_2nd = []

for idx, row in df.iterrows():
    # Only sequences with at least 2 digits
    if pd.notna(row['digit2']):
        rows_2nd.append({
            # Features: first digit and its sensor readings
            'digit1': row['digit1'],
            'acc1X': row['acc1X'], 'acc1Y': row['acc1Y'], 'acc1Z': row['acc1Z'],
            'gyro1X': row['gyro1X'], 'gyro1Y': row['gyro1Y'], 'gyro1Z': row['gyro1Z'],
            'rot1X': row['rot1X'], 'rot1Y': row['rot1Y'], 'rot1Z': row['rot1Z'],
            'mag1X': row['mag1X'], 'mag1Y': row['mag1Y'], 'mag1Z': row['mag1Z'],
            # Target
            'digit2': row['digit2']
        })

df_2nd = pd.DataFrame(rows_2nd)
df_2nd.to_csv(output_file_2nd, index=False)
print(f"2nd-digit dataset saved to {output_file_2nd}")
df_2nd.head()
df_2nd.info()

2nd-digit dataset saved to predict_2nd_digit.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5050 entries, 0 to 5049
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   digit1  5050 non-null   object 
 1   acc1X   5050 non-null   float64
 2   acc1Y   5050 non-null   float64
 3   acc1Z   5050 non-null   float64
 4   gyro1X  5050 non-null   float64
 5   gyro1Y  5050 non-null   float64
 6   gyro1Z  5050 non-null   float64
 7   rot1X   5050 non-null   float64
 8   rot1Y   5050 non-null   float64
 9   rot1Z   5050 non-null   float64
 10  mag1X   5050 non-null   float64
 11  mag1Y   5050 non-null   float64
 12  mag1Z   5050 non-null   float64
 13  digit2  5050 non-null   object 
dtypes: float64(12), object(2)
memory usage: 552.5+ KB


# Predict 3rd digit

In [115]:
output_file_3rd = "predict_3rd_digit.csv"
rows_3rd = []

for idx, row in df.iterrows():
    # Only sequences with at least 3 digits
    if pd.notna(row['digit3']):
        rows_3rd.append({
            # Features: first and second digits + readings
            'digit1': row['digit1'], 'digit2': row['digit2'],
            'acc1X': row['acc1X'], 'acc1Y': row['acc1Y'], 'acc1Z': row['acc1Z'],
            'gyro1X': row['gyro1X'], 'gyro1Y': row['gyro1Y'], 'gyro1Z': row['gyro1Z'],
            'rot1X': row['rot1X'], 'rot1Y': row['rot1Y'], 'rot1Z': row['rot1Z'],
            'mag1X': row['mag1X'], 'mag1Y': row['mag1Y'], 'mag1Z': row['mag1Z'],
            
            'acc2X': row['acc2X'], 'acc2Y': row['acc2Y'], 'acc2Z': row['acc2Z'],
            'gyro2X': row['gyro2X'], 'gyro2Y': row['gyro2Y'], 'gyro2Z': row['gyro2Z'],
            'rot2X': row['rot2X'], 'rot2Y': row['rot2Y'], 'rot2Z': row['rot2Z'],
            'mag2X': row['mag2X'], 'mag2Y': row['mag2Y'], 'mag2Z': row['mag2Z'],
            # Target
            'digit3': row['digit3']
        })

df_3rd = pd.DataFrame(rows_3rd)
df_3rd.to_csv(output_file_3rd, index=False)
print(f"3rd-digit dataset saved to {output_file_3rd}")
df_3rd.head()


3rd-digit dataset saved to predict_3rd_digit.csv


,digit1,digit2,acc1X,acc1Y,acc1Z,gyro1X,gyro1Y,gyro1Z,rot1X,rot1Y,...,gyro2X,gyro2Y,gyro2Z,rot2X,rot2Y,rot2Z,mag2X,mag2Y,mag2Z,digit3
0,7,0,0.903889,5.289334,8.043473,0.382707,0.074067,0.237626,0.170200,-0.233344,...,0.329867,-0.017868,-0.02138,0.187426,-0.257631,-0.722812,11.643750,29.643751,25.181252,1
1,7,0,0.970664,5.386789,7.320857,0.382707,0.074067,0.237626,0.170958,-0.132010,...,0.329867,-0.017868,-0.02138,0.249451,-0.269653,-0.765578,10.882029,30.705991,24.865260,1
2,7,0,0.847499,5.034560,8.165014,0.382707,0.074067,0.237626,0.171922,-0.275321,...,0.329867,-0.017868,-0.02138,0.171590,-0.236956,-0.650983,12.269135,29.974792,25.293949,1
3,7,0,0.964321,5.756778,7.570701,0.382707,0.074067,0.237626,0.151336,-0.195528,...,0.329867,-0.017868,-0.02138,0.083422,-0.235094,-0.654961,11.251435,27.528803,25.143211,1
4,7,0,0.915857,5.699441,8.634394,0.382707,0.074067,0.237626,0.221674,-0.297464,...,0.329867,-0.017868,-0.02138,0.213979,-0.217475,-0.709495,12.090919,28.032704,24.253187,1


# Predict 4th digit

In [116]:
output_file_4th = "predict_4th_digit.csv"
rows_4th = []

for idx, row in df.iterrows():
    # Only sequences with 4 digits
    if pd.notna(row['digit4']):
        rows_4th.append({
            # Features: first, second, third digits + readings
            'digit1': row['digit1'], 'digit2': row['digit2'], 'digit3': row['digit3'],
            'acc1X': row['acc1X'], 'acc1Y': row['acc1Y'], 'acc1Z': row['acc1Z'],
            'gyro1X': row['gyro1X'], 'gyro1Y': row['gyro1Y'], 'gyro1Z': row['gyro1Z'],
            'rot1X': row['rot1X'], 'rot1Y': row['rot1Y'], 'rot1Z': row['rot1Z'],
            'mag1X': row['mag1X'], 'mag1Y': row['mag1Y'], 'mag1Z': row['mag1Z'],

            'acc2X': row['acc2X'], 'acc2Y': row['acc2Y'], 'acc2Z': row['acc2Z'],
            'gyro2X': row['gyro2X'], 'gyro2Y': row['gyro2Y'], 'gyro2Z': row['gyro2Z'],
            'rot2X': row['rot2X'], 'rot2Y': row['rot2Y'], 'rot2Z': row['rot2Z'],
            'mag2X': row['mag2X'], 'mag2Y': row['mag2Y'], 'mag2Z': row['mag2Z'],

            'acc3X': row['acc3X'], 'acc3Y': row['acc3Y'], 'acc3Z': row['acc3Z'],
            'gyro3X': row['gyro3X'], 'gyro3Y': row['gyro3Y'], 'gyro3Z': row['gyro3Z'],
            'rot3X': row['rot3X'], 'rot3Y': row['rot3Y'], 'rot3Z': row['rot3Z'],
            'mag3X': row['mag3X'], 'mag3Y': row['mag3Y'], 'mag3Z': row['mag3Z'],
            # Target
            'digit4': row['digit4']
        })

df_4th = pd.DataFrame(rows_4th)
df_4th.to_csv(output_file_4th, index=False)
print(f"4th-digit dataset saved to {output_file_4th}")
df_4th.head()


4th-digit dataset saved to predict_4th_digit.csv


,digit1,digit2,digit3,acc1X,acc1Y,acc1Z,gyro1X,gyro1Y,gyro1Z,rot1X,...,gyro3X,gyro3Y,gyro3Z,rot3X,rot3Y,rot3Z,mag3X,mag3Y,mag3Z,digit4
0,2,1,4,2.392225,6.123233,7.140781,0.337503,-0.341932,0.168599,0.315441,...,-0.07422,0.067806,-0.023366,0.317786,0.105532,0.559608,9.750000,16.650002,8.775001,8
1,2,1,4,2.586351,6.616591,7.587349,0.337503,-0.341932,0.168599,0.420481,...,-0.07422,0.067806,-0.023366,0.293027,0.062829,0.606827,9.224184,17.606775,8.199797,8
2,2,1,4,2.333374,6.256945,6.596380,0.337503,-0.341932,0.168599,0.321324,...,-0.07422,0.067806,-0.023366,0.315834,0.060791,0.546017,9.454061,15.651741,8.787012,8
3,2,1,4,2.231158,5.504662,7.055258,0.337503,-0.341932,0.168599,0.243361,...,-0.07422,0.067806,-0.023366,0.310454,0.131279,0.568960,9.513384,16.372176,9.013618,8
4,2,1,4,2.393194,6.721773,7.316721,0.337503,-0.341932,0.168599,0.141956,...,-0.07422,0.067806,-0.023366,0.291830,0.085646,0.506426,10.685918,18.187982,9.503318,8


# Predict 3 digit Sequence

## Using 1 digit

In [117]:
import pandas as pd

input_file = "sequence_dataset.csv"

df_full = pd.read_csv(input_file, dtype=dtype_spec)
rows_3digit_1 = []
df= df_full[df_full['fullSequence'].astype(str).str.len() == 3].copy()

for idx, row in df.iterrows():
    # Only consider 3-digit sequences
    if pd.notna(row['digit3']):
        # Sequence after 1st digit
        seq_so_far = row['digit1']
        rows_3digit_1.append({
            'digit1': row['digit1'],
            'acc1X': row['acc1X'], 'acc1Y': row['acc1Y'], 'acc1Z': row['acc1Z'],
            'gyro1X': row['gyro1X'], 'gyro1Y': row['gyro1Y'], 'gyro1Z': row['gyro1Z'],
            'rot1X': row['rot1X'], 'rot1Y': row['rot1Y'], 'rot1Z': row['rot1Z'],
            'mag1X': row['mag1X'], 'mag1Y': row['mag1Y'], 'mag1Z': row['mag1Z'],
            'sequence': row['fullSequence']
        })

df_3digit_1 = pd.DataFrame(rows_3digit_1)
df_3digit_1.to_csv("3digit_predict_seq_after_1.csv", index=False)
df_3digit_1.head()


,digit1,acc1X,acc1Y,acc1Z,gyro1X,gyro1Y,gyro1Z,rot1X,rot1Y,rot1Z,mag1X,mag1Y,mag1Z,sequence
0,7,0.903889,5.289334,8.043473,0.382707,0.074067,0.237626,0.170200,-0.233344,-0.729515,9.675000,34.200000,32.925003,701
1,7,0.970664,5.386789,7.320857,0.382707,0.074067,0.237626,0.170958,-0.132010,-0.759705,9.035858,32.061615,32.488959,701
2,7,0.847499,5.034560,8.165014,0.382707,0.074067,0.237626,0.171922,-0.275321,-0.691215,10.316002,36.107039,30.166651,701
3,7,0.964321,5.756778,7.570701,0.382707,0.074067,0.237626,0.151336,-0.195528,-0.718030,8.868671,33.068256,33.040675,701
4,7,0.915857,5.699441,8.634394,0.382707,0.074067,0.237626,0.221674,-0.297464,-0.750522,10.452844,33.111958,31.971139,701


## Using 2 digits

In [118]:
rows_3digit_2 = []

for idx, row in df.iterrows():
    if pd.notna(row['digit3']):
        rows_3digit_2.append({
            'digit1': row['digit1'], 'digit2': row['digit2'],
            'acc1X': row['acc1X'], 'acc1Y': row['acc1Y'], 'acc1Z': row['acc1Z'],
            'gyro1X': row['gyro1X'], 'gyro1Y': row['gyro1Y'], 'gyro1Z': row['gyro1Z'],
            'rot1X': row['rot1X'], 'rot1Y': row['rot1Y'], 'rot1Z': row['rot1Z'],
            'mag1X': row['mag1X'], 'mag1Y': row['mag1Y'], 'mag1Z': row['mag1Z'],
            'acc2X': row['acc2X'], 'acc2Y': row['acc2Y'], 'acc2Z': row['acc2Z'],
            'gyro2X': row['gyro2X'], 'gyro2Y': row['gyro2Y'], 'gyro2Z': row['gyro2Z'],
            'rot2X': row['rot2X'], 'rot2Y': row['rot2Y'], 'rot2Z': row['rot2Z'],
            'mag2X': row['mag2X'], 'mag2Y': row['mag2Y'], 'mag2Z': row['mag2Z'],
            'sequence': row['fullSequence']
        })

df_3digit_2 = pd.DataFrame(rows_3digit_2)
df_3digit_2.to_csv("3digit_predict_seq_after_2.csv", index=False)
df_3digit_2.head()


,digit1,digit2,acc1X,acc1Y,acc1Z,gyro1X,gyro1Y,gyro1Z,rot1X,rot1Y,...,gyro2X,gyro2Y,gyro2Z,rot2X,rot2Y,rot2Z,mag2X,mag2Y,mag2Z,sequence
0,7,0,0.903889,5.289334,8.043473,0.382707,0.074067,0.237626,0.170200,-0.233344,...,0.329867,-0.017868,-0.02138,0.187426,-0.257631,-0.722812,11.643750,29.643751,25.181252,701
1,7,0,0.970664,5.386789,7.320857,0.382707,0.074067,0.237626,0.170958,-0.132010,...,0.329867,-0.017868,-0.02138,0.249451,-0.269653,-0.765578,10.882029,30.705991,24.865260,701
2,7,0,0.847499,5.034560,8.165014,0.382707,0.074067,0.237626,0.171922,-0.275321,...,0.329867,-0.017868,-0.02138,0.171590,-0.236956,-0.650983,12.269135,29.974792,25.293949,701
3,7,0,0.964321,5.756778,7.570701,0.382707,0.074067,0.237626,0.151336,-0.195528,...,0.329867,-0.017868,-0.02138,0.083422,-0.235094,-0.654961,11.251435,27.528803,25.143211,701
4,7,0,0.915857,5.699441,8.634394,0.382707,0.074067,0.237626,0.221674,-0.297464,...,0.329867,-0.017868,-0.02138,0.213979,-0.217475,-0.709495,12.090919,28.032704,24.253187,701


## Using 3 digits

In [119]:
rows_3digit_3 = []

for idx, row in df.iterrows():
    if pd.notna(row['digit3']):
        rows_3digit_3.append({
            'digit1': row['digit1'], 'digit2': row['digit2'], 'digit3': row['digit3'],
            'acc1X': row['acc1X'], 'acc1Y': row['acc1Y'], 'acc1Z': row['acc1Z'],
            'gyro1X': row['gyro1X'], 'gyro1Y': row['gyro1Y'], 'gyro1Z': row['gyro1Z'],
            'rot1X': row['rot1X'], 'rot1Y': row['rot1Y'], 'rot1Z': row['rot1Z'],
            'mag1X': row['mag1X'], 'mag1Y': row['mag1Y'], 'mag1Z': row['mag1Z'],

            'acc2X': row['acc2X'], 'acc2Y': row['acc2Y'], 'acc2Z': row['acc2Z'],
            'gyro2X': row['gyro2X'], 'gyro2Y': row['gyro2Y'], 'gyro2Z': row['gyro2Z'],
            'rot2X': row['rot2X'], 'rot2Y': row['rot2Y'], 'rot2Z': row['rot2Z'],
            'mag2X': row['mag2X'], 'mag2Y': row['mag2Y'], 'mag2Z': row['mag2Z'],

            'acc3X': row['acc3X'], 'acc3Y': row['acc3Y'], 'acc3Z': row['acc3Z'],
            'gyro3X': row['gyro3X'], 'gyro3Y': row['gyro3Y'], 'gyro3Z': row['gyro3Z'],
            'rot3X': row['rot3X'], 'rot3Y': row['rot3Y'], 'rot3Z': row['rot3Z'],
            'mag3X': row['mag3X'], 'mag3Y': row['mag3Y'], 'mag3Z': row['mag3Z'],

            'sequence': row['fullSequence']
        })

df_3digit_3 = pd.DataFrame(rows_3digit_3)
df_3digit_3.to_csv("3digit_predict_seq_after_3.csv", index=False)
df_3digit_3.head()


,digit1,digit2,digit3,acc1X,acc1Y,acc1Z,gyro1X,gyro1Y,gyro1Z,rot1X,...,gyro3X,gyro3Y,gyro3Z,rot3X,rot3Y,rot3Z,mag3X,mag3Y,mag3Z,sequence
0,7,0,1,0.903889,5.289334,8.043473,0.382707,0.074067,0.237626,0.170200,...,-0.013286,0.308029,-0.062461,0.206066,-0.260063,-0.721460,15.862500,20.662500,11.606251,701
1,7,0,1,0.970664,5.386789,7.320857,0.382707,0.074067,0.237626,0.170958,...,-0.013286,0.308029,-0.062461,0.145851,-0.350897,-0.751316,14.593916,19.414718,11.342061,701
2,7,0,1,0.847499,5.034560,8.165014,0.382707,0.074067,0.237626,0.171922,...,-0.013286,0.308029,-0.062461,0.191389,-0.255499,-0.823886,16.814924,21.963195,10.895139,701
3,7,0,1,0.964321,5.756778,7.570701,0.382707,0.074067,0.237626,0.151336,...,-0.013286,0.308029,-0.062461,0.137448,-0.285705,-0.762890,16.245528,19.320247,12.574039,701
4,7,0,1,0.915857,5.699441,8.634394,0.382707,0.074067,0.237626,0.221674,...,-0.013286,0.308029,-0.062461,0.184120,-0.284434,-0.781884,16.295595,20.980955,11.661559,701


# Predict 4 digit sequences

## Using 1 digit

In [120]:
rows_4digit_1 = []
df= df_full[df_full['fullSequence'].astype(str).str.len() == 4].copy()
for idx, row in df.iterrows():
    if pd.notna(row['digit4']):
        rows_4digit_1.append({
            'digit1': row['digit1'],
            'acc1X': row['acc1X'], 'acc1Y': row['acc1Y'], 'acc1Z': row['acc1Z'],
            'gyro1X': row['gyro1X'], 'gyro1Y': row['gyro1Y'], 'gyro1Z': row['gyro1Z'],
            'rot1X': row['rot1X'], 'rot1Y': row['rot1Y'], 'rot1Z': row['rot1Z'],
            'mag1X': row['mag1X'], 'mag1Y': row['mag1Y'], 'mag1Z': row['mag1Z'],
            'sequence': row['fullSequence']
        })

df_4digit_1 = pd.DataFrame(rows_4digit_1)
df_4digit_1.to_csv("4digit_predict_seq_after_1.csv", index=False)
df_4digit_1.head()


,digit1,acc1X,acc1Y,acc1Z,gyro1X,gyro1Y,gyro1Z,rot1X,rot1Y,rot1Z,mag1X,mag1Y,mag1Z,sequence
0,2,2.392225,6.123233,7.140781,0.337503,-0.341932,0.168599,0.315441,0.110244,0.555803,11.662500,17.925001,8.587501,2148
1,2,2.586351,6.616591,7.587349,0.337503,-0.341932,0.168599,0.420481,0.104929,0.539085,12.334544,17.712360,8.460039,2148
2,2,2.333374,6.256945,6.596380,0.337503,-0.341932,0.168599,0.321324,0.052869,0.507581,11.933361,17.174873,9.162962,2148
3,2,2.231158,5.504662,7.055258,0.337503,-0.341932,0.168599,0.243361,0.138985,0.725656,12.160768,18.784517,8.936834,2148
4,2,2.393194,6.721773,7.316721,0.337503,-0.341932,0.168599,0.141956,0.042154,0.657938,11.348705,18.662666,8.678149,2148


## Using 2 digits

In [121]:
rows_4digit_2 = []

for idx, row in df.iterrows():
    if pd.notna(row['digit4']):
        rows_4digit_2.append({
            'digit1': row['digit1'], 'digit2': row['digit2'],
            'acc1X': row['acc1X'], 'acc1Y': row['acc1Y'], 'acc1Z': row['acc1Z'],
            'gyro1X': row['gyro1X'], 'gyro1Y': row['gyro1Y'], 'gyro1Z': row['gyro1Z'],
            'rot1X': row['rot1X'], 'rot1Y': row['rot1Y'], 'rot1Z': row['rot1Z'],
            'mag1X': row['mag1X'], 'mag1Y': row['mag1Y'], 'mag1Z': row['mag1Z'],

            'acc2X': row['acc2X'], 'acc2Y': row['acc2Y'], 'acc2Z': row['acc2Z'],
            'gyro2X': row['gyro2X'], 'gyro2Y': row['gyro2Y'], 'gyro2Z': row['gyro2Z'],
            'rot2X': row['rot2X'], 'rot2Y': row['rot2Y'], 'rot2Z': row['rot2Z'],
            'mag2X': row['mag2X'], 'mag2Y': row['mag2Y'], 'mag2Z': row['mag2Z'],

            'sequence': row['fullSequence']
        })

df_4digit_2 = pd.DataFrame(rows_4digit_2)
df_4digit_2.to_csv("4digit_predict_seq_after_2.csv", index=False)
df_4digit_2.head()


,digit1,digit2,acc1X,acc1Y,acc1Z,gyro1X,gyro1Y,gyro1Z,rot1X,rot1Y,...,gyro2X,gyro2Y,gyro2Z,rot2X,rot2Y,rot2Z,mag2X,mag2Y,mag2Z,sequence
0,2,1,2.392225,6.123233,7.140781,0.337503,-0.341932,0.168599,0.315441,0.110244,...,-0.02306,0.089339,0.013134,0.330247,0.099312,0.553310,10.593750,17.156250,8.681251,2148
1,2,1,2.586351,6.616591,7.587349,0.337503,-0.341932,0.168599,0.420481,0.104929,...,-0.02306,0.089339,0.013134,0.338408,0.086919,0.514504,11.310962,17.658876,9.054142,2148
2,2,1,2.333374,6.256945,6.596380,0.337503,-0.341932,0.168599,0.321324,0.052869,...,-0.02306,0.089339,0.013134,0.374553,0.133252,0.504054,10.286874,17.641173,8.254160,2148
3,2,1,2.231158,5.504662,7.055258,0.337503,-0.341932,0.168599,0.243361,0.138985,...,-0.02306,0.089339,0.013134,0.268097,0.110875,0.435964,10.290277,15.461747,9.330785,2148
4,2,1,2.393194,6.721773,7.316721,0.337503,-0.341932,0.168599,0.141956,0.042154,...,-0.02306,0.089339,0.013134,0.336458,0.024012,0.521667,10.583784,15.485248,8.108727,2148


## Using 3 digits

In [122]:
rows_4digit_3 = []

for idx, row in df.iterrows():
    if pd.notna(row['digit4']):
        rows_4digit_3.append({
            'digit1': row['digit1'], 'digit2': row['digit2'], 'digit3': row['digit3'],
            'acc1X': row['acc1X'], 'acc1Y': row['acc1Y'], 'acc1Z': row['acc1Z'],
            'gyro1X': row['gyro1X'], 'gyro1Y': row['gyro1Y'], 'gyro1Z': row['gyro1Z'],
            'rot1X': row['rot1X'], 'rot1Y': row['rot1Y'], 'rot1Z': row['rot1Z'],
            'mag1X': row['mag1X'], 'mag1Y': row['mag1Y'], 'mag1Z': row['mag1Z'],

            'acc2X': row['acc2X'], 'acc2Y': row['acc2Y'], 'acc2Z': row['acc2Z'],
            'gyro2X': row['gyro2X'], 'gyro2Y': row['gyro2Y'], 'gyro2Z': row['gyro2Z'],
            'rot2X': row['rot2X'], 'rot2Y': row['rot2Y'], 'rot2Z': row['rot2Z'],
            'mag2X': row['mag2X'], 'mag2Y': row['mag2Y'], 'mag2Z': row['mag2Z'],

            'acc3X': row['acc3X'], 'acc3Y': row['acc3Y'], 'acc3Z': row['acc3Z'],
            'gyro3X': row['gyro3X'], 'gyro3Y': row['gyro3Y'], 'gyro3Z': row['gyro3Z'],
            'rot3X': row['rot3X'], 'rot3Y': row['rot3Y'], 'rot3Z': row['rot3Z'],
            'mag3X': row['mag3X'], 'mag3Y': row['mag3Y'], 'mag3Z': row['mag3Z'],

            'sequence': row['fullSequence']
        })

df_4digit_3 = pd.DataFrame(rows_4digit_3)
df_4digit_3.to_csv("4digit_predict_seq_after_3.csv", index=False)
df_4digit_3.head()


,digit1,digit2,digit3,acc1X,acc1Y,acc1Z,gyro1X,gyro1Y,gyro1Z,rot1X,...,gyro3X,gyro3Y,gyro3Z,rot3X,rot3Y,rot3Z,mag3X,mag3Y,mag3Z,sequence
0,2,1,4,2.392225,6.123233,7.140781,0.337503,-0.341932,0.168599,0.315441,...,-0.07422,0.067806,-0.023366,0.317786,0.105532,0.559608,9.750000,16.650002,8.775001,2148
1,2,1,4,2.586351,6.616591,7.587349,0.337503,-0.341932,0.168599,0.420481,...,-0.07422,0.067806,-0.023366,0.293027,0.062829,0.606827,9.224184,17.606775,8.199797,2148
2,2,1,4,2.333374,6.256945,6.596380,0.337503,-0.341932,0.168599,0.321324,...,-0.07422,0.067806,-0.023366,0.315834,0.060791,0.546017,9.454061,15.651741,8.787012,2148
3,2,1,4,2.231158,5.504662,7.055258,0.337503,-0.341932,0.168599,0.243361,...,-0.07422,0.067806,-0.023366,0.310454,0.131279,0.568960,9.513384,16.372176,9.013618,2148
4,2,1,4,2.393194,6.721773,7.316721,0.337503,-0.341932,0.168599,0.141956,...,-0.07422,0.067806,-0.023366,0.291830,0.085646,0.506426,10.685918,18.187982,9.503318,2148


## Using 4 digits

In [123]:
rows_4digit_4 = []

for idx, row in df.iterrows():
    if pd.notna(row['digit4']):
        rows_4digit_4.append({
            'digit1': row['digit1'], 'digit2': row['digit2'], 'digit3': row['digit3'], 'digit4': row['digit4'],
            'acc1X': row['acc1X'], 'acc1Y': row['acc1Y'], 'acc1Z': row['acc1Z'],
            'gyro1X': row['gyro1X'], 'gyro1Y': row['gyro1Y'], 'gyro1Z': row['gyro1Z'],
            'rot1X': row['rot1X'], 'rot1Y': row['rot1Y'], 'rot1Z': row['rot1Z'],
            'mag1X': row['mag1X'], 'mag1Y': row['mag1Y'], 'mag1Z': row['mag1Z'],

            'acc2X': row['acc2X'], 'acc2Y': row['acc2Y'], 'acc2Z': row['acc2Z'],
            'gyro2X': row['gyro2X'], 'gyro2Y': row['gyro2Y'], 'gyro2Z': row['gyro2Z'],
            'rot2X': row['rot2X'], 'rot2Y': row['rot2Y'], 'rot2Z': row['rot2Z'],
            'mag2X': row['mag2X'], 'mag2Y': row['mag2Y'], 'mag2Z': row['mag2Z'],

            'acc3X': row['acc3X'], 'acc3Y': row['acc3Y'], 'acc3Z': row['acc3Z'],
            'gyro3X': row['gyro3X'], 'gyro3Y': row['gyro3Y'], 'gyro3Z': row['gyro3Z'],
            'rot3X': row['rot3X'], 'rot3Y': row['rot3Y'], 'rot3Z': row['rot3Z'],
            'mag3X': row['mag3X'], 'mag3Y': row['mag3Y'], 'mag3Z': row['mag3Z'],

            'acc4X': row['acc4X'], 'acc4Y': row['acc4Y'], 'acc4Z': row['acc4Z'],
            'gyro4X': row['gyro4X'], 'gyro4Y': row['gyro4Y'], 'gyro4Z': row['gyro4Z'],
            'rot4X': row['rot4X'], 'rot4Y': row['rot4Y'], 'rot4Z': row['rot4Z'],
            'mag4X': row['mag4X'], 'mag4Y': row['mag4Y'], 'mag4Z': row['mag4Z'],

            'sequence': row['fullSequence']
        })

df_4digit_4 = pd.DataFrame(rows_4digit_4)
df_4digit_4.to_csv("4digit_predict_seq_after_4.csv", index=False)
df_4digit_4.head()


,digit1,digit2,digit3,digit4,acc1X,acc1Y,acc1Z,gyro1X,gyro1Y,gyro1Z,...,gyro4X,gyro4Y,gyro4Z,rot4X,rot4Y,rot4Z,mag4X,mag4Y,mag4Z,sequence
0,2,1,4,8,2.392225,6.123233,7.140781,0.337503,-0.341932,0.168599,...,0.003512,0.03543,0.020006,0.325562,0.103886,0.559829,7.893750,15.543751,8.812500,2148
1,2,1,4,8,2.586351,6.616591,7.587349,0.337503,-0.341932,0.168599,...,0.003512,0.03543,0.020006,0.355269,0.107705,0.604617,7.484246,14.406977,8.297676,2148
2,2,1,4,8,2.333374,6.256945,6.596380,0.337503,-0.341932,0.168599,...,0.003512,0.03543,0.020006,0.384002,0.221575,0.615188,7.056973,16.906431,8.120281,2148
3,2,1,4,8,2.231158,5.504662,7.055258,0.337503,-0.341932,0.168599,...,0.003512,0.03543,0.020006,0.364344,0.123480,0.496390,8.656761,14.142734,8.644696,2148
4,2,1,4,8,2.393194,6.721773,7.316721,0.337503,-0.341932,0.168599,...,0.003512,0.03543,0.020006,0.292047,0.065855,0.545997,8.025312,15.484764,9.193513,2148
